# Dados geográficos 

Nesse notebook, é realizada a limpeza dos dados geográficos das cidades. O intuito é ter uma base dos municipios que tem no Brasil, com informações como a densidade populacional, população estimada, área geográfica, localização geográfica e división por estados e regiões.

In [3]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
%matplotlib inline

## Dataframe Localização Geográfica

O primeiro dataframe a ser utilizado é a base com a latitude e longitude dos municipios brasileiros. Os dados são normalizados, criando uma coluna que elimina acentos e grafias diferentes.


In [4]:
PATH_MUNICIPIOS = '../dados/municipios/municipios.csv'
dfMunicipios = pd.read_csv(PATH_MUNICIPIOS, sep = ',', encoding = 'utf-8')
dfMunicipios['codigo_ibge'] = dfMunicipios['codigo_ibge'].astype(str)
dfMunicipios['municipioNormalizado'] = dfMunicipios['nome'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.lower()

In [5]:
dfMunicipios.head(5)

,codigo_ibge,nome,latitude,longitude,capital,codigo_uf,municipioNormalizado
0,5200050,Abadia de Goiás,-16.75730,-49.4412,0,52,abadia de goias
1,3100104,Abadia dos Dourados,-18.48310,-47.3916,0,31,abadia dos dourados
2,5200100,Abadiânia,-16.19700,-48.7057,0,52,abadiania
3,3100203,Abaeté,-19.15510,-45.4444,0,31,abaete
4,1500107,Abaetetuba,-1.72183,-48.8788,0,15,abaetetuba


# Dataframe Estimativa populacional

O próximo dataframe é a estimativa populacional para todos os municípios brasileiros no ano 2020

In [6]:
PATH_POPULACAO = '../dados/municipios/estimativa_populacao_2020.csv'
dfPopulacao = pd.read_csv(PATH_POPULACAO, sep = ',', encoding = 'utf-8')
dfPopulacao = dfPopulacao.rename(columns={'UF': 'uf', 'COD. UF': 'cod_uf', 'COD. MUNIC': 'cod_mun', 'NOME DO MUNICÍPIO': 'nome_mun', 'POPULAÇÃO ESTIMADA': 'populacao_estimada'})
dfPopulacao['codigo_ibge'] = dfPopulacao['cod_uf'].astype(str) + dfPopulacao['cod_mun'].astype(str).str.zfill(5)

In [7]:
dfPopulacao.head(5)

,uf,cod_uf,cod_mun,nome_mun,populacao_estimada,codigo_ibge
0,RO,11,15,'Alta Floresta D''Oeste',22728,1100015
1,RO,11,23,Ariquemes,109523,1100023
2,RO,11,31,Cabixi,5188,1100031
3,RO,11,49,Cacoal,85893,1100049
4,RO,11,56,Cerejeiras,16204,1100056


# Dataframe Área dos municípios

finalmente, o último dataframe contém a área dos municípios do Brasil. A área calculada está em km². 

In [25]:
PATH_AREA = '../dados/municipios/area.csv'
dfArea = pd.read_csv(PATH_AREA, sep = ';', encoding = 'utf-8')
dfArea = dfArea.rename(columns={'NM_UF_SIGLA': 'uf', 'CD_GCUF': 'cod_uf', 'CD_GCMUN': 'codigo_ibge', 'NM_MUN_2019': 'nome_mun', 'AR_MUN_2019': 'area_mun', 'NM_UF': 'estado'})
dfArea = dfArea.drop(columns = ['ID', 'cod_uf', 'uf', 'nome_mun'])
dfArea['codigo_ibge'] = dfArea['codigo_ibge'].astype(str)
dfArea['area_mun'] = dfArea['area_mun'].str.replace(',', '.').astype(float)
# dfArea['codigo_ibge'] = dfArea['cod_uf'].astype(str) + dfArea['cod_mun'].astype(str).str.zfill(5)

In [15]:
dfArea.head(5)

,estado,codigo_ibge,area_mun
0,Rondônia,1100015,7067.025
1,Rondônia,1100023,4426.571
2,Rondônia,1100031,1314.352
3,Rondônia,1100049,3792.892
4,Rondônia,1100056,2783.300


In [26]:
dfFinal = dfMunicipios.merge(dfPopulacao, on = 'codigo_ibge', how='inner').merge(dfArea, on = 'codigo_ibge', how='inner')
dfFinal = dfFinal.drop(columns=['nome_mun', 'cod_mun', 'cod_uf'])
dfFinal['densidade_populacional'] = dfFinal['populacao_estimada'] / dfFinal['area_mun']

In [27]:
PATH_TOWRITE = '../dados/municipios/municipios_processada.csv' 
dfFinal.to_csv(PATH_TOWRITE, sep=';', encoding='utf-8')